<a href="https://colab.research.google.com/github/Prachi194agrawal/ipl_ai_agent/blob/main/ipl_agent_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
5!pip install kaggle
from google.colab import files

# 1) Upload kaggle.json from your Kaggle account
files.upload()  # select kaggle.json

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 2) Download IPL dataset (example: jayfaldu/ipl-dataset)
!kaggle datasets download -d jayfaldu/ipl-dataset -p data
!unzip data/ipl-dataset.zip -d data


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/jayfaldu/ipl-dataset
License(s): DbCL-1.0
  0% 0.00/1.20M [00:00<?, ?B/s]
100% 1.20M/1.20M [00:00<00:00, 734MB/s]
Archive:  data/ipl-dataset.zip
  inflating: data/deliveries.csv     
  inflating: data/matches.csv        


In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv("/content/data/matches.csv")
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
import pandas as pd
import numpy as np

df = matches.copy()

# Keep essential columns (adjust names to your file)
cols = [
    "team1", "team2", "toss_winner", "toss_decision",
    "venue", "winner"
]
df = df[cols].dropna()

# Target: 1 if team1 == winner else 0
df["team1_win"] = (df["winner"] == df["team1"]).astype(int)

# Encode toss decision as first/second batting indicator
df["toss_bat"] = (df["toss_decision"].str.lower() == "bat").astype(int)

# Label encode teams and venues
from sklearn.preprocessing import LabelEncoder

for col in ["team1", "team2", "toss_winner", "venue"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

X = df[["team1", "team2", "toss_winner", "toss_bat", "venue"]]
y = df["team1_win"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


Model: logreg
Accuracy: 0.5364238410596026
              precision    recall  f1-score   support

           0       0.55      0.90      0.68        84
           1       0.38      0.07      0.12        67

    accuracy                           0.54       151
   macro avg       0.47      0.49      0.40       151
weighted avg       0.48      0.54      0.44       151

Model: rf
Accuracy: 0.5364238410596026
              precision    recall  f1-score   support

           0       0.57      0.70      0.63        84
           1       0.47      0.33      0.39        67

    accuracy                           0.54       151
   macro avg       0.52      0.52      0.51       151
weighted avg       0.52      0.54      0.52       151

Model: xgb
Accuracy: 0.5099337748344371
              precision    recall  f1-score   support

           0       0.55      0.62      0.58        84
           1       0.44      0.37      0.40        67

    accuracy                           0.51       151
   mac

In [ ]:
# OPTION 1: Kaggle CLI (jayfaldu/ipl-dataset)
from google.colab import files
files.upload()  # upload kaggle.json first

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d jayfaldu/ipl-dataset -p data
!unzip -o data/ipl-dataset.zip -d data

# Explore filenames
import os
for root, _, files_list in os.walk("data"):
    for f in files_list:
        print(os.path.join(root, f))


Saving AI_Agent_IPL_Task_Updated.pdf to AI_Agent_IPL_Task_Updated.pdf
Dataset URL: https://www.kaggle.com/datasets/jayfaldu/ipl-dataset
License(s): DbCL-1.0
ipl-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  data/ipl-dataset.zip
  inflating: data/deliveries.csv     
  inflating: data/matches.csv        
data/matches.csv
data/deliveries.csv
data/ipl-dataset.zip


In [ ]:
# Colab: basic setup
!pip install kaggle xgboost scikit-learn pandas numpy joblib

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import joblib


In [ ]:
matches = pd.read_csv("data/matches.csv")  # adjust filename if needed
matches.head()
matches.columns


Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')

In [ ]:
# Keep only relevant, non-null rows
cols = ["team1", "team2", "toss_winner", "toss_decision", "venue", "winner"]
df = matches[cols].dropna().copy()

# Remove matches with no winner (tie/NR)
df = df[df["winner"] != "no result"]

# Target: 1 if team1 wins else 0
df["team1_win"] = (df["winner"] == df["team1"]).astype(int)

# Toss decision: bat = 1, field = 0
df["toss_bat"] = (df["toss_decision"].str.lower() == "bat").astype(int)

from sklearn.preprocessing import LabelEncoder

team_le = LabelEncoder()
venue_le = LabelEncoder()

# Fit team encoder on all team columns together to keep mapping consistent
all_teams = pd.concat([df["team1"], df["team2"], df["toss_winner"]]).unique()
team_le.fit(all_teams)

df["team1_enc"] = team_le.transform(df["team1"])
df["team2_enc"] = team_le.transform(df["team2"])
df["toss_winner_enc"] = team_le.transform(df["toss_winner"])

df["venue_enc"] = venue_le.fit_transform(df["venue"])

X = df[["team1_enc", "team2_enc", "toss_winner_enc", "toss_bat", "venue_enc"]]
y = df["team1_win"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape


((601, 5), (151, 5))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

models = {
    "logreg": LogisticRegression(max_iter=2000),
    "rf": RandomForestClassifier(
        n_estimators=500, max_depth=10, min_samples_split=4,
        random_state=42, n_jobs=-1
    ),
    "xgb": XGBClassifier(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=7,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1,
    ),
}

results = {}
for name, clf in models.items():
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    proba = clf.predict_proba(X_test)[:, 1]
    acc = accuracy_score(y_test, preds)
    auc = roc_auc_score(y_test, proba)
    print(f"=== {name} ===")
    print("Accuracy:", acc)
    print("ROC-AUC:", auc)
    print(classification_report(y_test, preds))
    results[name] = (acc, auc)


=== logreg ===
Accuracy: 0.5364238410596026
ROC-AUC: 0.44367448471926085
              precision    recall  f1-score   support

           0       0.55      0.90      0.68        84
           1       0.38      0.07      0.12        67

    accuracy                           0.54       151
   macro avg       0.47      0.49      0.40       151
weighted avg       0.48      0.54      0.44       151

=== rf ===
Accuracy: 0.5364238410596026
ROC-AUC: 0.4840085287846482
              precision    recall  f1-score   support

           0       0.57      0.64      0.61        84
           1       0.47      0.40      0.44        67

    accuracy                           0.54       151
   macro avg       0.52      0.52      0.52       151
weighted avg       0.53      0.54      0.53       151

=== xgb ===
Accuracy: 0.543046357615894
ROC-AUC: 0.5056858564321252
              precision    recall  f1-score   support

           0       0.58      0.63      0.61        84
           1       0.48     

In [ ]:
best_model = models["xgb"]
best_model


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=600, n_jobs=-1,
              num_parallel_tree=None, ...)

In [ ]:
import os, json, joblib

os.makedirs("models", exist_ok=True)
os.makedirs("artifacts", exist_ok=True)
os.makedirs("rag_corpus", exist_ok=True)

# Save model
joblib.dump(best_model, "models/ipl_xgb_model.pkl")

# Save encoders
joblib.dump(team_le, "artifacts/team_encoder.pkl")
joblib.dump(venue_le, "artifacts/venue_encoder.pkl")

# Save simple meta info (optional)
meta = {
    "n_matches": int(len(df)),
    "features": list(X.columns),
    "model": "xgboost",
    "test_accuracy": float(results["xgb"][0]),
    "test_roc_auc": float(results["xgb"][1]),
}
with open("artifacts/meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Artifacts saved.")


Artifacts saved.
